In [304]:
import pychrome #in command line type - pip install pychrome
import pandas as pd #pandas library
import numpy as np
from selenium import webdriver #pip install selenium
from webdriver_manager.chrome import ChromeDriverManager #pip install webdriver-manager
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [305]:
options = webdriver.ChromeOptions()
options.add_argument("--remote-debugging-port=8000")

In [306]:
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\Madhav\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache


c:\users\madhav\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [307]:
dev_tools = pychrome.Browser(url="http://localhost:8000")

In [308]:
tab = dev_tools.list_tab()[0]

In [309]:
tab.start()

True

In [310]:
city_list = ['New Delhi','Hyderabad','Chennai','Mumbai','Kolkata','Bangalore']
letters=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
# letters = ["क","ख","ग","घ","ड","च","छ","ज","झ","ञ","ट","ठ","ड","ढ","ण","त","थ","द","ध","न","प","फ","ब","भ","म","य","र","ल","व","श","ष","स","ह","ष","र","ञ","र","क़","ग़","ख़","ज़","फ़","ढ़","ड़"]

In [311]:
df = pd.DataFrame(letters,columns=['Letter'])

In [312]:
df['Word_1'] = np.empty(len(letters))
df['Word_2'] = np.empty(len(letters))
df['Word_3'] = np.empty(len(letters))
df['Word_4'] = np.empty(len(letters))
df['Word_5'] = np.empty(len(letters))
df['Word_6'] = np.empty(len(letters))
df['Word_7'] = np.empty(len(letters))
df['Word_8'] = np.empty(len(letters))
df['Word_9'] = np.empty(len(letters))
df['Word_10'] = np.empty(len(letters))

In [313]:
df.index = df['Letter']

In [314]:
df.drop(['Letter'],axis=1,inplace=True)

In [315]:
coordinates = pd.DataFrame(city_list,columns=['Cities'])

In [316]:
latitude = [28.644800,17.387140,13.067439,19.076090,22.5499978,12.972442]

In [317]:
longitude = [77.216721,78.491684,80.237617,72.877426,88.333332,77.580643]

In [318]:
coordinates['Latitude'] = latitude
coordinates['Longitude'] = longitude

In [319]:
coordinates

,Cities,Latitude,Longitude
0,New Delhi,28.644800,77.216721
1,Hyderabad,17.387140,78.491684
2,Chennai,13.067439,80.237617
3,Mumbai,19.076090,72.877426
4,Kolkata,22.549998,88.333332
5,Bangalore,12.972442,77.580643


In [320]:
for i in range(len(coordinates)):
    df_copy = df.copy()
    driver.get("https://www.google.co.in/search?q=a")
    time.sleep(3)
    tab.call_method("Network.enable", _timeout=20)
    tab.call_method("Browser.grantPermissions",permissions=['geolocation'])
    tab.call_method("Emulation.setGeolocationOverride",latitude=coordinates.iloc[i,1],longitude=coordinates.iloc[i,2]
                    ,accuracy=100)
    time.sleep(20)
    bodyText = driver.find_element_by_tag_name('body').text
    if 'Use precise location' in bodyText:
        driver.find_element_by_xpath("//a[text()='Use precise location']").click()
    else:
        driver.find_element_by_xpath("//a[text()='Update location']").click()
    time.sleep(3)
    driver.get("https://www.google.co.in")
    time.sleep(2)
    search_field = driver.find_element_by_name("q")
    for j in range(len(letters)):
        search_field.send_keys(letters[j])
        time.sleep(2)
        for loop in range(1,11):
            try:
                li_items = driver.find_element_by_xpath('//*/ul/li[%d]/div/div[2]/div[1]' %(loop)).text
                df_copy.iloc[j,loop-1] = li_items
            except NoSuchElementException:
                time.sleep(10)
                try:
                    li_items = driver.find_element_by_xpath('//*/ul/li[%d]/div/div[2]/div[1]' %(loop)).text
                    df_copy.iloc[j,loop-1] = li_items
                except NoSuchElementException:
                    for inner_loop in range(loop-1,10):
                        df_copy.iloc[j,inner_loop] = None
                    break
        driver.find_element_by_name("q").clear()
    file_name = "trial_data/" +str(coordinates.iloc[i,0])+"_data.csv"
    df_copy.to_csv(file_name,index=['Letter'])

                  Word_1             Word_2              Word_3  \
Letter                                                            
क़                कोरोना   कारगिल विजय दिवस  कोरोना वायरस टिप्स   
ग़                 ग़ज़ल    ग़ालिब की शायरी       ग़ालिब के शेर   
ख़                   खेल      खुजली का इलाज      खतरा खतरा खतरा   
ज़            ज़ से शब्द  ज़ी न्यूज़ समाचार      ज़िन्दगी शायरी   
फ़      फ़ैज़ अहमद फ़ैज़         फ़ से शब्द      फ़िज़ा का अर्थ   

                   Word_4                         Word_5           Word_6  \
Letter                                                                      
क़              कृष्ण भजन            कृष्ण भगवान की आरती        कुंभ राशि   
ग़      ग़दर एक प्रेम कथा                           None             None   
ख़                 खरबूजा                           खजूर              खबर   
ज़               ज़ी टीवी  ज़ी न्यूज़ बिहार झारखण्ड लाइव  ज़ी न्यूज़ लाइव   
फ़               फ़िल्में      फ़ूड डिपार्टमेंट राजस्थान          फ़

                  Word_1              Word_2          Word_3  \
Letter                                                         
क़      कारगिल विजय दिवस  कोरोना वायरस टिप्स       कृष्ण भजन   
ग़                 ग़ज़ल     ग़ालिब की शायरी   ग़ालिब के शेर   
ख़                   खेल      खतरा खतरा खतरा          खरबूजा   
ज़            ज़ से शब्द      ज़िन्दगी शायरी        ज़ी टीवी   
फ़      फ़ैज़ अहमद फ़ैज़          फ़ से शब्द  फ़िज़ा का अर्थ   

                               Word_4                     Word_5  \
Letter                                                             
क़                कृष्ण भगवान की आरती        क्या तुम शर्मीली हो   
ग़                  ग़दर एक प्रेम कथा                       None   
ख़                               खजूर                        खबर   
ज़      ज़ी न्यूज़ बिहार झारखण्ड लाइव          ज़ी न्यूज़ समाचार   
फ़                           फ़िल्में  फ़ूड डिपार्टमेंट राजस्थान   

                    Word_6                                      Word_7  \


websocket exception
Traceback (most recent call last):
  File "c:\users\madhav\anaconda3\lib\site-packages\pychrome\tab.py", line 121, in _recv_loop
    message_json = self._ws.recv()
  File "c:\users\madhav\anaconda3\lib\site-packages\websocket\_core.py", line 314, in recv
    opcode, data = self.recv_data()
  File "c:\users\madhav\anaconda3\lib\site-packages\websocket\_core.py", line 331, in recv_data
    opcode, frame = self.recv_data_frame(control_frame)
  File "c:\users\madhav\anaconda3\lib\site-packages\websocket\_core.py", line 344, in recv_data_frame
    frame = self.recv_frame()
  File "c:\users\madhav\anaconda3\lib\site-packages\websocket\_core.py", line 378, in recv_frame
    return self.frame_buffer.recv_frame()
  File "c:\users\madhav\anaconda3\lib\site-packages\websocket\_abnf.py", line 361, in recv_frame
    self.recv_header()
  File "c:\users\madhav\anaconda3\lib\site-packages\websocket\_abnf.py", line 309, in recv_header
    header = self.recv_strict(2)
  File "c:\user